In [1]:
import cv2
import csv

In [2]:
filenames = []
labels = []
with open('Labels.csv') as index:
    index = csv.reader(index)
    for r in index:
        filenames.append(r[0])
        labels.append(r[1])
        
del filenames[0]
del labels[0]

In [3]:
import numpy as np

In [4]:
labels_hot = []
for x in labels:
    if x == 'Positive':
        labels_hot.append([1,0,0])
    if x == 'Neutral':
        labels_hot.append([0,1,0])
    if x == 'Negative':
        labels_hot.append([0,0,1])

In [ ]:
lb_idx = 0
final_label=[]
f_idx = 0
classfier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
color = (0,255,0)
for file in filenames:
    filename = './presidential_videos/' + file
    cap = cv2.VideoCapture(filename)
    count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        if not success:
            break
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faceRects = classfier.detectMultiScale(gray, scaleFactor=1.5,minNeighbors=8, minSize=(32,32))
        
        if len(faceRects)>0:
            for fr in faceRects:
                x,y,w,h = fr
#                 cv2.rectangle(frame,(x-20,y-20), (x+w+10,y+h+10), color, 1)
                image = frame[y:y+h, x:x+w]
                im_name = './data/' + str(f_idx)+'.jpg'
                final_label.append(labels[lb_idx])
                f_idx += 1
                cv2.imwrite(im_name, image)
                count += 1
#                 cv2.imshow('frame', frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        if count >= 30:
            break
    lb_idx += 1
#     break
# cv2.destroyAllWindows()
cap.release()

In [ ]:
print(len(final_label))

In [ ]:
import os

image_size = 64
def resize_img(image, height = image_size, width = image_size):
    top,bottom, left, right = (0,0,0,0)
    
    h,w,_=image.shape
    
    longest_edge = max(h,w)
    
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass
    
    BLACK = [0,0,0]
    constant = cv2.copyMakeBorder(image, top,bottom, left,right, cv2.BORDER_CONSTANT,value = BLACK)
    
    return cv2.resize(constant, (height,width))

In [ ]:
images = []

count = 0
img_name = str(count) +'.jpg'

for i in range(len(final_label)):
    image = cv2.imread('./data/'+img_name)
    if image is None:
        del final_label[i]
    else:
        image = resize_img(image, image_size, image_size)
        images.append(image)
        
print(len(images))
print(len(final_label))
        

In [ ]:
images = np.array(images, dtype='float')
final_label = np.array(final_label)

print(images.shape)
print(final_label.shape)



In [ ]:
import keras
from sklearn.model_selection import train_test_split
from keras.preprocesing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPoling2D
from keras.optimizers import SGD
from keras.models import load_model
from keras.import backend as K

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(images, final_label, test_size = 0.3, random_state = random.randint(0,100))
print(test_labels)

In [ ]:
image_Channels = 3

In [ ]:
if K.image_data_format == 'channel_first':
    train_images = train_images.reshape(train_images.shape[0],img_channels, image_size, image_size)
    test_images = test_images.reshape(test_images.shape[0],img_channels, image_size, image_size)
    self.input_shape = (img_channels, image_size, image_size)
else:
    train_images = train_images.reshape(train_images.shape[0], image_size, image_size, img_channels)
    test_images = test_images.reshape(test_images.shape[0], image_size, image_size, img_channels)
    self.input_shape = (image_size, image_size, img_channels)

In [ ]:
print(train_images.shape[0],'train samples')
print(test_images.shape[0], 'test samples')

In [ ]:
train_images /= 255
test_images /= 255

